# Shared Task: Critical thinking generation
## Loadind Data


In [ ]:
pip install -q social_agents/

In [ ]:
pip show social_agents

In [ ]:
import tqdm
# Reading the data
import json
import pandas as pd
import social_agents

from social_agents.utils import get_st_data


for key, line in tqdm.tqdm(get_st_data("sample").items()):
    print(key)

    print(line['intervention'])
    input_text = line['intervention']
    break


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

## Prerequisites
env, LLM, ...

In [ ]:
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("MISTRAL_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "shared_task_critical_questions"


##  1. Zero Shot LLM to start with o3-mini

In [ ]:

from social_agents.graph_tools import BasicCQModel
model_name = "o3-mini"

basic_agent = BasicCQModel(llm_name = model_name, temperature=None ) #interrupt_before=['human_feedback'], checkpointer=memory)


display(Image(basic_agent.graph.get_graph(xray=1).draw_mermaid_png()))

# RUN
# basic_agent.run_experiment()



### Evaulate 

- Run shell 

```shell

python3 eval_scripts/evaluation.py \
    --metric similarity \
    --input_path data_splits/validation.json \
    --submission_path output/elbaff_experiment/output_gpt-4o-mini-2024-07-18social_n2_Te_Sr.json \
    --threshold 0.6 

```

- OUTPUT : output/output_o3-mini_temperatureNA._eval_similarity_06.json



**Overall count**

| **Questions Labels** |  **#**  | **ratio** |
|:--------------------|-------:|---------:|
| useful               | **329** |  **0,59** |
| unhelpful            |      63 |      0,11 |
| Invalid              |       8 |      0,01 |
| Not able to evaluate |     158 |      0,28 |
| **Total**            | **558** |         1 |


**Overall count within each argument**

| **n/3 useful questions per arg** | **# of arguments** | **ratio** |
|:-------------------------------:|-------------------:|:---------:|
|                               0/3 |                 17 |      0,10 |
|                               1/3 |                 51 |      0,27 |
|                               2/3 |             **76** |      0,40 |
|                               3/3 |                 42 |      0,23 |
| **Total**                       |            **186** |         1 |

## Social Agents


In [7]:
import pandas as pd
import os

from social_agents.graph_tools import  CQSTAbstractAgent, SocialAgentBuilder

EXPERIMENT_SETTINGS_FILE_PATH = "output/elbaff_experiment/experiment_settings.csv"
override = False
if not os.path.exists(EXPERIMENT_SETTINGS_FILE_PATH) or override:
    print("generating exp settings file")
    exps_df = SocialAgentBuilder._generate_experiment_settings()
    exps_df.to_csv(EXPERIMENT_SETTINGS_FILE_PATH, index=False)

experiment_settings = pd.read_csv(EXPERIMENT_SETTINGS_FILE_PATH)
experiment_settings.head()

# exp_row = experiment_settings.iloc[20].to_dict()
# exp_row

,traits,strategies,rounds,number_of_agents,has_debate,thread_id,experiment_name
0,"('overconfident',)","('reflect',)",1,1,False,bd1b0527-d5c2-4d72-ad58-2208a9b7844e,{llm_name}social_n1_To_Sr
1,"('overconfident',)","('reflect', 'reflect')",2,1,False,fffc391f-ce92-4d15-b622-f98b8a94d4db,{llm_name}social_n1_To_Srr
2,"('overconfident',)","('reflect', 'reflect', 'reflect')",3,1,False,ffea6ff4-81d3-4abb-9c7c-c093e203ec07,{llm_name}social_n1_To_Srrr
3,"('easy_going',)","('reflect',)",1,1,False,c93f8004-a694-48a4-91f5-2f86ab6463a7,{llm_name}social_n1_Te_Sr
4,"('easy_going',)","('reflect', 'reflect')",2,1,False,ea3c9fa1-854e-48d1-b917-df05ec873709,{llm_name}social_n1_Te_Srr


In [ ]:
import ast
from social_agents import objects
import ast
from tqdm import tqdm
from IPython.display import Image, display

llm_name = "gpt-4o-mini-2024-07-18"# "o3-mini-2025-01-31"
llm_name_short = "gpt-4o-mini_"
temperature = 0.8
for idx, row in tqdm(experiment_settings.iterrows(), total=len(experiment_settings)):
    exp_name = row["experiment_name"].format(llm_name= llm_name_short)
    if os.path.exists(f"{SocialAgentBuilder.ROOT_FOLDER}output_{exp_name}.json"):
        print("Experiment ", exp_name, "already done!")
        continue
    if "easy_going" in list(ast.literal_eval(row["traits"])):
        print("="*20)
        print(row.to_dict)
        print("EXPERIMENT NAME: ", exp_name)
        print("="*20)
        
        social_agent = SocialAgentBuilder(
            model_thread_id=row["thread_id"],
            llm_name = llm_name,
            llm_num = row["number_of_agents"],
            experiment_name= exp_name,
            temperature=temperature,
            collaborative_strategy=list(ast.literal_eval(row["strategies"])),
            agent_trait_lst=list(ast.literal_eval(row["traits"])))
        display(Image(social_agent.graph.get_graph(xray=1).draw_mermaid_png()))

        social_agent.run_experiment(data_type = "validation", save= True)
        print(f'finished {exp_name}')




  4%|▎         | 4/113 [00:00<00:01, 93.77it/s]


Experiment  gpt-4o-mini_social_n1_Te_Sr already done!
<bound method Series.to_dict of traits                                   ('easy_going',)
strategies                        ('reflect', 'reflect')
rounds                                                 2
number_of_agents                                       1
has_debate                                         False
thread_id           ea3c9fa1-854e-48d1-b917-df05ec873709
experiment_name               {llm_name}social_n1_Te_Srr
Name: 4, dtype: object>
EXPERIMENT NAME:  gpt-4o-mini_social_n1_Te_Srr
Initializing LLM gpt-4o-mini-2024-07-18
Building Agentic Graph
building workflow


ValueError: Node `moderator_question` already present.

In [ ]:
display(Image(social_agent.graph.get_graph(xray=1).draw_mermaid_png()))
